In [126]:
def read_instance(file):
    liste=open(file, "r")# ouvrir la liste des instances
    nb_obj = liste.readline()# lire le nombre des objets
    cap_max = int(liste.readline())# lire le nombre des objets
    list_prof = []
    list_vol = []
    for j in range(int(nb_obj)): #boucle pour lire et résoudre les instances une par une
        line = liste.readline()#lire le nom
        l=line.split()
        list_vol.append(int(l[0]))
        list_prof.append(int(l[1]))
    return (cap_max, list_vol, list_prof)

In [127]:
def heu_const(cap_max, list_vol, list_prof):
    nb_obj = len(list_vol)
    list_densite = []
    for i in range(nb_obj): #boucle pour lire et résoudre les instances une par une
        list_densite.append(list_prof[i]/list_vol[i])
    tri_densite = []
    for i in range(nb_obj):
        m = max(list_densite)
        k = list_densite.index(m)
        tri_densite.append(k)
        list_densite[k]=-1
    cap = 0
    sol = []
    i = 0
    prof = 0
    while (cap<=cap_max)and(i<nb_obj):
        index = tri_densite[i]
        if (cap +list_vol[index])<=cap_max:
            cap += list_vol[index]
            prof += list_prof[index]
            sol.append(index)
        i+=1
    return (sol, cap, prof)


In [128]:
def heu_const_aleatoire(cap_max, list_vol, list_prof):
    nb_obj = len(list_vol)
    list_densite = []
    for i in range(nb_obj): #boucle pour lire et résoudre les instances une par une
        list_densite.append(list_prof[i]/list_vol[i])
    tri_densite = []
    for i in range(nb_obj):
        m = max(list_densite)
        k = list_densite.index(m)
        tri_densite.append(k)
        list_densite[k]=-1
    from random import randint
    #choisir aléatoirement le premier element de la solution
    #puis continuer la construction de la solution en ajoutant les éléments les plus dense   
    first_elt = randint(0,nb_obj-1)
    cap = list_vol[first_elt]
    sol = [first_elt]
    i = 0
    k = tri_densite.index(first_elt)
    prof = list_prof[first_elt]
    while (cap<=cap_max)and(i<nb_obj)and (i!=k):
        index = tri_densite[i]
        if (cap +list_vol[index])<=cap_max:
            cap += list_vol[index]
            prof += list_prof[index]
            sol.append(index)
        i+=1
    return (sol, cap, prof)

In [148]:
"""La liste tabou contient les positions déjà explorées. Ils sont conservées dans une file FIFO d'une taille donnée, 
qui est un paramètre ajustable de l'heuristique.
la solution [41,8] est la même [8,41] donc il faut convertir les solution en des ensembles car dans les listes l'ordre des
objets est pris en considération"""
#une solution explorée s et s n'est pas dans list_tabou alors list_tabou.append(set(s))

def tabou(s,list_tabou):
    if set(s) in list_tabou:
        return True
    list_tabou.append(set(s))
    return False

In [171]:
def capacite(s,list_vol):
    c = 0
    for i in s:
        c += list_vol[i]
    return c
def profit(s,list_prof):
    p = 0
    for i in s:
        p += list_prof[i]
    return p
def voisin(s,cap_max,list_vol, list_prof):
    """on supprime un element de la solution s et on ajoute un autre element parmi les elemenst qui ne sont pas dans 
    la solution s. Puis on repete avec tous les elements de s""" 
    v = []
    nb_obj = len(list_vol)
    c = capacite(s,list_vol)
    for i in s :
        s1 = s.copy()
        s1.remove(i)
        cap_voisin = c - list_vol[i]
        #on supprime un element de la solution s
        for k in range(nb_obj):
            cap_voisin = cap_voisin + list_vol[k]
            if (k not in s) and (cap_voisin <= cap_max) :
                #on ajoute un autre element parmi les elemenst qui ne sont pas dans la solution s.
                s1 = s1+[k]
        v.append(s1)
        #on repete avec tous les elements de s
    return v
def recherche_local(s0,cap_max, list_vol, list_prof):
    list_tabou = []
    s = s0
    nb_obj = len(list_vol)
    amelioration = True
    while amelioration:
        amelioration = False
        prof_max = 0
        smax = []
        for i in voisin(s,cap_max,list_vol, list_prof):
                if profit(i,list_prof)>prof_max:
                    smax = i;
                    prof_max = profit(i,list_prof)
        if prof_max > profit(s,list_prof):
            s = smax;
            amelioration = True
    return (s, capacite(s,list_vol), profit(s,list_prof))


In [168]:
def GRASP(cap_max, list_vol, list_prof):
    nbr_iteration = 10
    prof_max = 0
    sol_optimal = []
    for i in range (nbr_iteration):
        s0, _, prof0 = heu_const_aleatoire(cap_max, list_vol, list_prof)
        s_prime, cap,prof = recherche_local(s0,cap_max, list_vol, list_prof)
        if prof > prof_max:
            sol_optimal = s_prime
            prof_max = prof
    return (sol_optimal, cap, prof_max)     

In [169]:
instance = read_instance('test100-SC(1).txt')
print(heu_const(instance[0], instance[1], instance[2]))
print(heu_const_aleatoire(instance[0], instance[1], instance[2]))

([21], 101, 91)
([15, 35], 101, 81)


In [170]:
import numpy as np
import csv
import time

with open("MyResult.csv", "w") as f_write: # créer et ouvrir le fichier Resultat.csv
    writer = csv.writer(f_write, delimiter=";")# spécifier le délimiteur des colonnes
    writer.writerow( ["instance","objectif recherche local","temps(s)","objectif GRASP","temps(s)"] )#écrire les noms des colonnes dans Resultat.csv
    files=open("all.txt", "r")# ouvrir la liste des instances
    nb_inst = files.readline()# lire le nombre des instances à résoudre
    print("nb_inst=", int(nb_inst))#affichage du nombre des instances à résoudre pour vérification
    for j in range(int(nb_inst)): #boucle pour lire et résoudre les instances une par une
        file = files.readline()#lire le nom de l'instance j à résoudre
       #pour toutes les ligne dans all.txt, le nom de l'instance est sauvgardé dans la chaine inst suivit 
       #du caractère de fin de ligne \n sauf pour la dernière ligne d'ou il faut enlever ce \n
        if j < int(nb_inst): #vérifier si ce n'est pas la dernière ligne dans all.txt
            print(file)
            file=file[:-1]#prebdre toute la chaine sauf le dernier caractère (le \n)
            cap_max, list_vol, list_prof = read_instance(file)
            t0 = time.time()
            s0, _, _ = heu_const(cap_max, list_vol, list_prof)
            sol_rech_loc , vol_sol_rech_loc, prof_sol_rech_loc = recherche_local(s0,cap_max, list_vol, list_prof)
            t1 = time.time()
            sol_GRASP, vol_sol_GRASP, prof_sol_GRASP = GRASP(cap_max, list_vol, list_prof)       
            t2 = time.time()
            writer.writerow( [file,int(prof_sol_rech_loc), round(t1-t0,3), int(prof_sol_GRASP), round(t2-t1,3)] )#ecrire le nom de linstance, l'objectif et le temps de résolution dans resultat.csv
    files.close()# fermer le fichier liste des instances


nb_inst= 25
test100-SC(1).txt

95 101
[0]
[46]
[20]
[25]
[88]
95 101
[0]
[]
95 101
[0]
[]
95 101
[0]
[]
95 101
[0]
[]
[35]
[15]
[30]
[29]
95 101
[0]
[]
95 101
[0]
[]
test100-SC(2).txt

14 101
70 101
[0, 1]
14 101
70 101
[0, 1]
75 101
[80, 0]
54 101
[84, 0]
66 101
[27, 0]
63 101
[77, 0]
14 101
70 101
[0, 1]
77 101
[74, 0]
52 101
[8, 0]
99 101
[91, 0]
25 101
81 101
[12, 0, 1]
84 101
[0, 1]
[91]
14 101
70 101
[0, 1]
14 101
70 101
[0, 1]
72 101
[2, 0]
55 101
[88, 0]
99 101
[91, 0]
25 101
81 101
[30, 0, 1]
84 101
[0, 1]
[91]
test100-SC(3).txt

84 101
[0]
[6]
[57]
84 101
[0]
[]
84 101
[0]
84 101
[0]
[]
[6]
[76]
[73]
[8]
84 101
[0]
84 101
[0]
[]
[41]
[54]
[87]
[95]
test100-SC(4).txt

55 101
[0]
[39]
66 101
[15, 0]
55 101
[0]
[4]
79 101
[40, 0]
55 101
[0]
55 101
[0]
[]
[78]
69 101
[43, 0]
55 101
[0]
55 101
[0]
[7]
81 101
[96, 0]
[30]
90 101
[34, 0]
test100-SC(5).txt

25 101
87 101
[0, 1]
[64]
48 101
[66, 0]
25 101
87 101
[0, 1]
80 101
[15, 0]
70 101
[88, 0]
25 101
87 101
[0, 1]
83 101
[8, 0]
6

[6, 22, 75, 27]
345 400
[385, 22, 75, 27, 0]
345 400
[385, 6, 75, 27, 0]
345 400
[385, 6, 22, 27, 0]
[385, 6, 22, 75]
[6, 22, 75, 197]
345 400
[267, 22, 75, 197, 0]
345 400
[267, 6, 75, 197, 0]
345 400
[267, 6, 22, 197, 0]
[267, 6, 22, 75]
275 400
384 400
[6, 22, 0, 1]
257 400
366 400
[555, 22, 0, 1]
257 400
366 400
[555, 6, 0, 1]
[22, 0, 1]
[6, 0, 1]
[6, 22, 1]
[6, 22, 0]
366 400
[6, 22, 14, 0]
345 400
[1817, 22, 14, 0]
345 400
[1817, 6, 14, 0]
364 400
[1817, 6, 22, 0]
test2000-SC(5).txt

409 494
471 494
[558, 574, 635, 17, 0, 1]
409 494
471 494
[76, 574, 635, 17, 0, 1]
409 494
471 494
[76, 558, 635, 17, 0, 1]
409 494
471 494
[76, 558, 574, 17, 0, 1]
465 494
[76, 558, 574, 635, 0]
465 494
[76, 558, 574, 635, 0]
395 494
457 494
[842, 558, 574, 635, 0, 1]
395 494
457 494
[842, 76, 574, 635, 0, 1]
395 494
457 494
[842, 76, 558, 635, 0, 1]
395 494
457 494
[842, 76, 558, 574, 0, 1]
[76, 558, 574, 635, 31]
409 494
471 494
[163, 558, 574, 635, 31, 0, 1]
409 494
471 494
[163, 76, 574, 635, 31